In [1]:
import pandas as pd
df = pd.read_csv("./data/정치.csv")

In [2]:
n = 13
content = df['content'][n]
title = df['title'][n] 

import re
def pre(content) :
    content = content[68:]
    src = re.search(r'(\..{0,30}기자[ \,])' ,content)
    if src :
        content = content[:src.span()[0]+1]
    #print(len(content))
    ad_re = re.compile('(\[.{0,20}\])|(\(.{0,20}\))|(▶.*)')
    content = ad_re.sub('', content)
    return content

content = pre(content)
content

'북미 정상회담 성과와 과제 북한 <조선중앙통신>은 김정은 국무위원장과 도널드 트럼프 미국 대통령이 2월28일 베트남 하노이 메트로폴 호텔에서 단독회담과 확대회담을 했다고 1일 보도했다. 평양/조선중앙통신 연합뉴스  합의 없이 끝난 하노이 2차 북-미 정상회담은 “한반도의 항구적이고 공고한 평화체제 구축”이라는 긴 여정의 새 출발점일 수밖에 없다. 김정은 국무위원장과 도널드 트럼프 대통령의 ‘하노이 담판’ 결과는 새 모색의 실마리이자 추동력이 될 ‘성과’와 함께 3차 정상회담을 성취하려면 반드시 풀어야 할 ‘과제’도 드러냈다.  성과① 확인된 ‘상황 관리’ 능력 관계 악화 막으려 상대 배려 후퇴 않는다는 ‘역진 방지’ 공감 “두 정상이 회담 이후 서로를 배려하며 상황 관리에 애를 쓰고 있다. 이번에 확인된 가장 큰 성과의 하나다.” “두 정상이 ‘판을 깰 생각이 없다’는 뜻을 분명히 하고 있다.” 고위 소식통은 3일 “북-미 정상이 ‘역진 방지’에 공감한다는 게 확인되고 있다”며 “후퇴만 하지 않는다면 다시 전진할 수 있다”고 기대 섞인 분석을 내놨다. 실제 김 위원장은 하노이 회담에서 “생산적인 대화들을 계속 이어나가기로 했다”는 메시지를 <노동신문>을 통해 내놨다. 트럼프 대통령도 회담 직후 기자회견에서 “김 위원장과 관계가 매우 돈독하다. 궁극적으로 서로 합의할 수 있을 것”이라고 강조했다. 성과② 드러난 ‘거래의 조건’ 비핵화·제재 완화 수위 확인 불확실성 제거로 새 회담 기반  “북한이 영변 핵시설을 통째로 포기할 생각이 있다는 사실이 처음으로 확인됐다. 영변이 폐기되면 나머지 비핵화 과정이 빨라질 수 있다. 가장 큰 성과다.” “불확실성이 제거되고 모든 쟁점이 분명해졌다. 3차 정상회담에서 더 높은 차원의 합의가 이뤄질 기반이 마련됐다.”(조성렬 전 국가안보전략연구원 선임연구위원) 이종석 전 장관은 “북쪽이 영변 핵시설 영구 폐기, 더구나 미국 전문가의 사찰·검증과 북-미 양국 기술자의 공동작업 폐기 방안을 제안한 사실이 매우 중요하다”며 “새로운 

In [ ]:
import collections
from itertools import takewhile
from nltk import ngrams
from konlpy.tag import Mecab
from collections import defaultdict

def extract_ngram(content, ngram_n=5, cutoff = 2, group_by_pos_li = ['NNP', 'NNG', 'SN', 'SH', 'SL', 'NNBC']) : 
    mecab = Mecab()
    nouns = [token[0] for token in mecab.pos(content) if token[1] in group_by_pos_li] 
    ngrams_li = [ngram for n in range(ngram_n, 1, -1) for ngram in ngrams(nouns, n)] #bigram ~ n-gram 까지 생성
    counter = collections.Counter(ngrams_li) #ngram을 빈도별로 집계
    ngram_counter =  dict(takewhile(lambda i: i[1] >= cutoff, counter.most_common())) #빈도 cutoff 이상 ngram 추출
    
    # 빈도 우선으로 이미 등록된 ngram을 포함하고 있는 경우는 제외
    dct = defaultdict()
    for item in ngram_counter :        
        chk = next(( False for key in dct.keys() if (set(item).issubset(key)) | (set(key).issubset(item))), True)
        if chk : dct[item] = ngram_counter[item]
        
    return list(dct.keys())

ngram = extract_ngram(content, ngram_n=7, cutoff=2)
ngram

In [5]:
import nltk
from nltk.tokenize import sent_tokenize
from konlpy.tag import Mecab
import networkx as nx
import math
import matplotlib.pyplot as plt
import numpy as np

class TextRank :
    def __init__(self, text) :
        self.tokens = self.pos_tagging(text)

    def pos_tagging(self, content):
        def group_by_pos(tokens, join_char='') :
            return (join_char.join([t[0] for t in tokens]), 'NNP' if len(set([t[1] for t in tokens])) > 2 else tokens[-1][1] )

        def gen_tokens(tokens, join_char='', group_by_pos_li = ['NNP', 'NNG', 'SN', 'SH', 'SL', 'NNBC'], stop_pos=[]) :
            last_token = tokens[0]

            ret = []
            li = []
            for t in tokens :
                if t[1] in group_by_pos_li : li.append(t)
                else :
                    if len(li) > 0 : ret.append(group_by_pos(li, join_char))
                    if t[1] not in stop_pos : ret.append(t)
                    li=[]
            if len(li) > 0 : ret.append(group_by_pos(li, join_char))

            return ret

        mecab = Mecab()
        stop_pos = ['IC', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JC','JX'
                    , 'XR', 'SF', 'SE', 'SSO', 'SSC', 'SC', 'SY', 'EC', 'EF', 'ETN', 'ETM', 'XSV', 'XSA', 'XSN', 'XPN']
        ret = []
        for s in sent_tokenize(content) :
            #ret.append((s, sent_li))
            sent_li= []
            sent_li.append([(w, gen_tokens(mecab.pos(w), stop_pos=stop_pos)) for w in s.split()])

            #for w in s.split() : sent_li.append((w, gen_tokens(mecab.pos(w))))
            ret.append((s, sent_li))
            #ret.append((s, [t for t in gen_tokens(sent_li, join_char= ' ') if t[1] not in stop_pos]))

        return ret
        
    def keywords(self, n=10) :        
        tokens = [t for s in self.tokens for w in s[1] for t in w]
        nodes = [k for t in tokens for k in t[1] if (k[1][0] in ['N', 'V']) & (len(k[0])>1)]
        tokens = [k for t in tokens for k in t[1]]
        
        def connect(nodes, tokens) :            
            window_size = 5 # coocurrence를 판단하기 위한 window 사이즈 설정

            edges = []
            for window_start in range(0,(len(tokens)-window_size+1)):
                window = tokens[window_start:window_start+window_size]            
                #edges.append([(window[i], window[j]) for i in range(window_size) for j in range(window_size) if ( (i > j) & (window[i] in nodes) & (window[j] in nodes))])

                for i in range(window_size) :
                    for j in range(window_size) : 
                        if (i > j) & (window[i] in nodes) & (window[j] in nodes) :            
                            edges.append((window[i], window[j]))
            return edges

        graph=nx.diamond_graph()
        graph.clear() #처음 생성시 graph에 garbage node가 남아있어 삭제
        graph.add_nodes_from(list(set(nodes))) #node 등록
        graph.add_edges_from(connect(nodes, tokens)) #edge 연결
        scores = nx.pagerank(graph) #pagerank 계산
        rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
        return rank[:n]
    
    def print_keywords(self, n=10) :   
        print("Keyword : ")
        for k in self.keywords(n) :
            print("{} - {}".format(k[0][0], k[1]))
    
    def summarize(self, n=3) :
        #자카드 유사도 계산
        def jaccard_similarity(query, document):
            intersection = set(query).intersection(set(document))
            union = set(query).union(set(document))
            return len(intersection)/len(union)

        # 문장간 유사도 측정 (BoW를 활용 코사인 유사도 측정)
        def sentence_similarity(sentence1, sentence2):
            mecab = Mecab()
            
            sentence1 = mecab.morphs(sentence1[0])#[t[0] for s in sentence1[1][0] for t in s[1] if t[1][0] in ['N','V'] ] 
            sentence2 = mecab.morphs(sentence2[0])#.split()#[t[0] for s in sentence2[1][0] for t in s[1] if t[1][0] in ['N','V'] ]
            #print(sentence1)
            return jaccard_similarity(sentence1, sentence2)

        def sentences(doc):
            return [s[0].strip() for s in doc]

        def connect(doc):
            return [(start[0].strip(),end[0].strip() ,sentence_similarity(start, end)) 
                    for start in doc for end in doc if start is not end]
        
        graph=nx.diamond_graph()
        graph.clear() #처음 생성시 graph에 garbage node가 남아있어 삭제
        graph.add_nodes_from(sentences(self.tokens)) #node 등록
        graph.add_weighted_edges_from(connect(self.tokens)) #edge 연결
        scores = nx.pagerank(graph) #pagerank 계산
        #print(scores)
        rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
        return rank[:n]
    
    def print_summarize(self, n=3) :   
        print("Summarize : ")
        for s in self.summarize(n) :

            print("{} - {}".format(s[0], s[1]))
        

In [6]:
print("원문")
print(title)
print(content[:700])
print()
print("---- krayon ----")
tr = TextRank(pre(content))
tr.print_keywords(5)
print()
tr.print_summarize(3)

print("\n---- gensim ----")
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize
print(keywords(content, ratio = 0.1).split('\n'))
print(summarize(content))

원문
 ‘북·미 판 깰 의도 없다’ 확인했지만…교착 장기화 막아야 
북미 정상회담 성과와 과제 북한 <조선중앙통신>은 김정은 국무위원장과 도널드 트럼프 미국 대통령이 2월28일 베트남 하노이 메트로폴 호텔에서 단독회담과 확대회담을 했다고 1일 보도했다. 평양/조선중앙통신 연합뉴스  합의 없이 끝난 하노이 2차 북-미 정상회담은 “한반도의 항구적이고 공고한 평화체제 구축”이라는 긴 여정의 새 출발점일 수밖에 없다. 김정은 국무위원장과 도널드 트럼프 대통령의 ‘하노이 담판’ 결과는 새 모색의 실마리이자 추동력이 될 ‘성과’와 함께 3차 정상회담을 성취하려면 반드시 풀어야 할 ‘과제’도 드러냈다.  성과① 확인된 ‘상황 관리’ 능력 관계 악화 막으려 상대 배려 후퇴 않는다는 ‘역진 방지’ 공감 “두 정상이 회담 이후 서로를 배려하며 상황 관리에 애를 쓰고 있다. 이번에 확인된 가장 큰 성과의 하나다.” “두 정상이 ‘판을 깰 생각이 없다’는 뜻을 분명히 하고 있다.” 고위 소식통은 3일 “북-미 정상이 ‘역진 방지’에 공감한다는 게 확인되고 있다”며 “후퇴만 하지 않는다면 다시 전진할 수 있다”고 기대 섞인 분석을 내놨다. 실제 김 위원장은 하노이 회담에서 “생산적인 대화들을 계속 이어나가기로 했다”는 메시지를 <노동신문>을 통해 내놨다. 트럼프 대통령도 회담 직후 기자회견에서 “김 위원장과 관계가 매우 돈독하다. 궁극적으로 서로 합의할 수 있을 것”이라고 강조했다. 성과② 드러난 ‘거래의 조건’ 비핵화·

---- krayon ----
Keyword : 
회담 - 0.01808343233691793
미국 - 0.015118195432661727
트럼프 - 0.011476840513489921
하노이 - 0.010921497119051597
비핵화 - 0.010897973452043906

Summarize : 
이번에 확인된 가장 큰 성과의 하나다.” “두 정상이 ‘판을 깰 생각이 없다’는 뜻을 분명히 하고 있다.” 고위 소식통은 3일 “북-미 정상이 ‘역진 방지

In [ ]:
def daily_issue(date) : 
    for content in sub_df[sub_df.date==date]['content'][:100] :
        content = pre(content)
        tr = TextRank(content)
        tr.print_keywords(5)
        tr.print_summarize()
        print()
        
sub_df = df[df.category2=="정치일반"]
daily_issue('2020-03-03')  

In [ ]:

        for rank in textrank(processed_text, 10) :
            if rank[0] not in d.keys() : 
                d[rank[0]] = rank[1]   
            else :
                d[rank[0]] = d[rank[0]] + rank[1]

    n = 10
    cnt = 0
    print(date)
    li = []
    for item in {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)} :
        li.append(item)
        cnt = cnt + 1
        if n == cnt :
            break
    print(li)
    return li

In [ ]:

a = []

for d in pd.date_range('2020-03-03','2020-03-17') :
    dstr = d.strftime('%Y-%m-%d')
    bb = daily_issue(d.strftime('%Y-%m-%d'))
    #print(bb)
    a.append((dstr, bb))
    

In [ ]:
from collections import defaultdict
dct = defaultdict()

for d in a :
    for i in d[1] :
        if i not in dct.keys() :
            dct[i] = 1
        else :
            dct[i] = dct[i] + 1

print("공통이슈")            
for k in dct.keys() :
    if dct[k] > 5:
        print(k)
        
print("\n날짜별이슈")                        
for d in a :
    print(d[0])
    for i in d[1] :
        if dct[i] < 5 :
            print(i)